In [1]:
import numpy as np
import json

### Import File

In [2]:
features_data = open("data/data_cleaned_head1000/textfeatures_head1000", "r")
target_data = open("data/data_cleaned_head1000/starstarget_head1000", "r")
#features_data = open("data/data_cleaned/textfeatures0", "r")
#target_data = open("data/data_cleaned/starstarget0", "r")
text = [line for line in features_data]
target = [int(line.split()[0]) for line in target_data]


The target contains number from 1 to 5, according to the rating (stars). In order to classify the commment in "positive" (+1) or "negative" (0), we convey to choose positive if stars are grater or equal to 3 and negative elsewhere.



In [3]:
target = [ 1 if target[i] >=3 else 0 for i in range(len(target ))]
dataset = [[text[i], target[i]] for i in range(len(target))]

## Rebalancing the Classes

With a simple counting we realize that the two classes have completly different sizes. There are much more positive comments than negative ones. The ratio is about $1/3$

In [4]:
target_pos = target.count(1)
target_neg = target.count(0)
ratio = float(target_neg) / target_pos

In [5]:
import random

def rebalance(dataset_to_reb, length, ratio): 
    reb_dataset = []
    for i in range(length):
        if dataset_to_reb[i][1] == 0:
            reb_dataset.append(dataset_to_reb[i])
        else:
            rnd = random.random()
            if rnd < ratio:
                reb_dataset.append(dataset_to_reb[i])
    return reb_dataset

reb_dataset = rebalance(dataset, len(target), ratio)

reb_text = [ tx for [tx,tg] in reb_dataset]
reb_target = [ tg for [tx,tg] in reb_dataset]


In [6]:
# First selection of stop_words

default_stop_words = ['all', 'six', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'not', 'fifty', 'four', 'own', 'through', 'yourselves', 'go', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'with', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'under', 'ours', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'whether', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'several', 'hereafter', 'always', 'who', 'cry', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'two', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'eg', 'some', 'back', 'up', 'namely', 'towards', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its', 'everything', 'behind', 'un', 'above', 'between', 'it', 'neither', 'seemed', 'ever', 'across', 'she', 'somehow', 'be', 'we', 'full', 'never', 'sixty', 'however', 'here', 'otherwise', 'were', 'whereupon', 'nowhere', 'although', 'found', 'alone', 're', 'along', 'fifteen', 'by', 'both', 'about', 'last', 'would', 'anything', 'via', 'many', 'could', 'thence', 'put', 'against', 'keep', 'etc', 'amount', 'became', 'ltd', 'hence', 'onto', 'or', 'con', 'among', 'already', 'co', 'afterwards', 'formerly', 'within', 'seems', 'into', 'others', 'while', 'whatever', 'except', 'down', 'hers', 'everyone', 'done', 'least', 'another', 'whoever', 'moreover', 'couldnt', 'throughout', 'anyhow', 'yourself', 'three', 'from', 'her', 'few', 'together', 'top', 'there', 'due', 'been', 'next', 'anyone', 'eleven', 'much', 'call', 'therefore', 'interest', 'then', 'thru', 'themselves', 'hundred', 'was', 'sincere', 'empty', 'more', 'himself', 'elsewhere', 'mostly', 'on', 'fire', 'am', 'becoming', 'hereby', 'amongst', 'else', 'part', 'everywhere', 'too', 'herself', 'former', 'those', 'he', 'me', 'myself', 'made', 'twenty', 'these', 'bill', 'cant', 'us', 'until', 'besides', 'nevertheless', 'below', 'anywhere', 'nine', 'can', 'of', 'your', 'toward', 'my', 'something', 'and', 'whereafter', 'whenever', 'give', 'almost', 'wherever', 'is', 'describe', 'beforehand', 'herein', 'an', 'as', 'itself', 'at', 'have', 'in', 'seem', 'whence', 'ie', 'any', 'fill', 'again', 'hasnt', 'inc', 'thereby', 'thin', 'no', 'perhaps', 'latter', 'meanwhile', 'when', 'detail', 'same', 'wherein', 'beside', 'also', 'that', 'other', 'take', 'which', 'becomes', 'you', 'if', 'nobody', 'see', 'though', 'may', 'after', 'upon', 'most', 'hereupon', 'eight', 'but', 'serious', 'nothing', 'such', 'why', 'a', 'off', 'whereby', 'third', 'i', 'whole', 'noone', 'sometimes', 'well', 'amoungst', 'yours', 'their', 'rather', 'without', 'so', 'five', 'the', 'first', 'whereas', 'once']
list_word_to_remove = ['not']
my_stop_words = [word for word in default_stop_words if word not in list_word_to_remove]

my_stop_words.append('ve') 
my_stop_words.append('ll')
my_stop_words.append('got')
my_stop_words.append('know')
my_stop_words.append('15')
my_stop_words.append('30')
my_stop_words.append('20')
my_stop_words.append('50')

In [7]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reb_text, reb_target, test_size=0.2, random_state=42)

/Users/Yak52/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Industrialize your model: Pipeline 

In [8]:
%%time 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier

from sklearn import pipeline



# Pipeline for Decision Tree
def my_pipeline_dt(thres):
    nsw = np.loadtxt("data/stop_words/my_stop_words_thr_%.2f.txt" %thres, dtype=str)
    pipe = pipeline.Pipeline([
                            ('count_vectorizer', CountVectorizer(stop_words = list(nsw), max_df = 0.9, min_df = 10)),
                            #('count_vectorizer', CountVectorizer(stop_words = my_stop_words, max_df = 0.9, min_df = 10)),
                            ('tf_idf',TfidfTransformer()),
                            ('model',DecisionTreeClassifier(criterion='gini', 
                                                            splitter='best', 
                                                            max_depth=10, 
                                                            min_samples_split=2, 
                                                            min_samples_leaf=1, 
                                                            min_weight_fraction_leaf=0.0, 
                                                            max_features=None, 
                                                            random_state=1, 
                                                            max_leaf_nodes=None, 
                                                            min_impurity_split=1e-07, 
                                                            class_weight=None, 
                                                            presort=False))
                         ])
    return pipe


CPU times: user 17.2 ms, sys: 14 ms, total: 31.2 ms
Wall time: 46.4 ms


In [9]:
mypipe = my_pipeline_dt(0.5).fit(X_train, y_train)
score_train_pipe = mypipe.score(X_train, y_train)
score_test_pipe = mypipe.score(X_test, y_test)
print "%f \t %f \t" %(score_train_pipe, score_test_pipe)


0.890080 	 0.702128 	


# Random Forest Classifier

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

from sklearn import pipeline


def my_pipeline_rf(thres):
    nsw = np.loadtxt("data/stop_words/my_stop_words_thr_%.2f.txt" %thres, dtype=str)
    pipe = pipeline.Pipeline([
                            ('count_vectorizer', CountVectorizer(stop_words = list(nsw), max_df = 0.9, min_df = 10)),
                            #('count_vectorizer', CountVectorizer(stop_words = my_stop_words, max_df = 0.9, min_df = 10)),
                            ('tf_idf',TfidfTransformer()),
                            ('model',RandomForestClassifier(n_estimators=200, 
                                                               criterion='gini',
                                                               max_depth=2, 
                                                               min_samples_split=2, 
                                                               min_samples_leaf=1, 
                                                               min_weight_fraction_leaf=0.0, 
                                                               max_features='auto', 
                                                               max_leaf_nodes=None, 
                                                               min_impurity_split=1e-07, 
                                                               bootstrap=True, 
                                                               oob_score=False, 
                                                               n_jobs=1, 
                                                               random_state=None, 
                                                               verbose=0, 
                                                               warm_start=False, 
                                                               class_weight=None))
                         ])
    return pipe

CPU times: user 6.97 ms, sys: 4.3 ms, total: 11.3 ms
Wall time: 15.1 ms


# AdaBoost Classifier

An AdaBoost classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.

In [11]:
from sklearn.ensemble import AdaBoostClassifier

def my_pipeline_AdaBoost(thres):
    nsw = np.loadtxt("data/stop_words/my_stop_words_thr_%.2f.txt" %thres, dtype=str)
    pipe = pipeline.Pipeline([
                            ('count_vectorizer', CountVectorizer(stop_words = list(nsw), max_df = 0.9, min_df = 10)),
                            #('count_vectorizer', CountVectorizer(stop_words = my_stop_words, max_df = 0.9, min_df = 10)),
                            ('tf_idf',TfidfTransformer()),
                            ('model',AdaBoostClassifier(base_estimator=None, 
                                                           n_estimators=50, 
                                                           learning_rate=1.0, 
                                                           algorithm='SAMME.R', 
                                                           random_state=None))
                         ])
    return pipe

# Linear Model: Logistic Regression



In [12]:
from sklearn.linear_model import LogisticRegression

def my_pipeline_logreg(thres):
    nsw = np.loadtxt("data/stop_words/my_stop_words_thr_%.2f.txt" %thres, dtype=str)
    pipe = pipeline.Pipeline([
                            ('count_vectorizer', CountVectorizer(stop_words = list(nsw), max_df = 0.9, min_df = 10)),
                            #('count_vectorizer', CountVectorizer(stop_words = my_stop_words, max_df = 0.9, min_df = 10)),
                            ('tf_idf',TfidfTransformer()),
                            ('model',LogisticRegression(penalty='l2', 
                                                           dual=False, 
                                                           tol=0.0001, 
                                                           C=1.0, 
                                                           fit_intercept=True, 
                                                           intercept_scaling=1, 
                                                           class_weight=None, 
                                                           random_state=None, 
                                                           solver='liblinear', 
                                                           max_iter=100,
                                                           multi_class='ovr',
                                                           verbose=0, 
                                                           warm_start=False, 
                                                           n_jobs=1))
                         ])
    return pipe

# Parameter Optimisation: Grid Search for Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from pandas import DataFrame

pipe = my_pipeline_rf(0.5)

gs = GridSearchCV(
                    pipe,
                    {
                        "count_vectorizer__min_df": range(1,10)+range(10,30,5),
                        "model__max_depth": range(1,20)+range(30,100,10)
                    },
                    cv=2,  # 5-fold cross validation
                    n_jobs=16,  # run each hyperparameter in one of two parallel jobs
                    scoring="accuracy" # what could happen selecting "precision" as scoring measure?
                )

gs.fit(X_train, y_train)

np.save("rf_grid_results.npy", gs.cv_results_)
np.save("rf_grid_bestparameters.npy", gs.best_params_)

# Parameter Optimisation: Grid Search for Logistic Regression

In [14]:
from sklearn.model_selection import GridSearchCV
from pandas import DataFrame

pipe = my_pipeline_lr(0.5)

gs = GridSearchCV(
                    pipe,
                    {
                        "count_vectorizer__min_df": range(1,10)+range(10,30,5),
                        "model__C": [0.1, 0.5, 1.0, 3.0, 5.0, 10, 20, 50, 80]
                    },
                    cv=2,  # 5-fold cross validation
                    n_jobs=16,  # run each hyperparameter in one of two parallel jobs
                    scoring="accuracy" # what could happen selecting "precision" as scoring measure?
                )

gs.fit(X_train, y_train)

np.save("lr_grid_results.npy", gs.cv_results_)
np.save("lr_grid_bestparameters.npy", gs.best_params_)

{'count_vectorizer__min_df': 2, 'model__max_depth': 80}


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_count_vectorizer__min_df,param_model__max_depth,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,1.068869,0.186368,0.584450,0.702432,1,1,"{u'count_vectorizer__min_df': 1, u'model__max_...",337,0.614973,0.709677,0.553763,0.695187,0.030202,0.001224,0.030605,0.007245
1,1.036088,0.173914,0.635389,0.801622,1,2,"{u'count_vectorizer__min_df': 1, u'model__max_...",319,0.647059,0.806452,0.623656,0.796791,0.013968,0.000719,0.011701,0.004830
2,1.075050,0.194080,0.683646,0.868639,1,3,"{u'count_vectorizer__min_df': 1, u'model__max_...",222,0.700535,0.870968,0.666667,0.866310,0.068205,0.007957,0.016934,0.002329
3,1.138940,0.187187,0.672922,0.882080,1,4,"{u'count_vectorizer__min_df': 1, u'model__max_...",253,0.689840,0.897849,0.655914,0.866310,0.064726,0.003754,0.016963,0.015770
4,1.041751,0.192088,0.686327,0.884653,1,5,"{u'count_vectorizer__min_df': 1, u'model__max_...",209,0.700535,0.860215,0.672043,0.909091,0.001550,0.005761,0.014246,0.024438
5,1.020226,0.182668,0.686327,0.914180,1,6,"{u'count_vectorizer__min_df': 1, u'model__max_...",209,0.684492,0.903226,0.688172,0.925134,0.010428,0.006972,0.001840,0.010954
6,1.040145,0.179445,0.715818,0.930294,1,7,"{u'count_vectorizer__min_df': 1, u'model__max_...",61,0.721925,0.930108,0.709677,0.930481,0.014741,0.000986,0.006124,0.000187
7,1.068545,0.180240,0.680965,0.946323,1,8,"{u'count_vectorizer__min_df': 1, u'model__max_...",239,0.684492,0.924731,0.677419,0.967914,0.010634,0.002979,0.003536,0.021592
8,1.035372,0.180315,0.721180,0.959778,1,9,"{u'count_vectorizer__min_df': 1, u'model__max_...",42,0.705882,0.956989,0.736559,0.962567,0.005584,0.001026,0.015338,0.002789
9,1.035758,0.178486,0.707775,0.957090,1,10,"{u'count_vectorizer__min_df': 1, u'model__max_...",114,0.721925,0.951613,0.693548,0.962567,0.021173,0.001817,0.014188,0.005477


In [23]:
type(range(1,20))

list

In [24]:
range(1,20)+range(30,100,10)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 30,
 40,
 50,
 60,
 70,
 80,
 90]